## TensorFlow and Deep Learning Malaysia Meetup Sept 2018

Recommender System using TensorFlow

In [1]:
# Importing tensorflow
import tensorflow as tf

# Importing some more libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as MSE

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# reading the ratings data
ratings = pd.read_csv('ml-1m/ratings.dat', sep="::", header = None, engine='python')

# Lets pivot the data to get it at a user level
ratings_pivot = pd.pivot_table(ratings[[0,1,2]], values=2, index=0, columns=1 ).fillna(0)

# creating train and test sets
X_train, X_test = train_test_split(ratings_pivot, train_size=0.8)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
# Deciding how many nodes wach layer should have
n_nodes_inpl = 3706  
n_nodes_hl1  = 256  
n_nodes_outl = 3706

# first hidden layer has 784*32 weights and 32 biases
hidden_1_layer_vals = {'weights':tf.Variable(tf.random_normal ([n_nodes_inpl+1,n_nodes_hl1]))}

# first hidden layer has 784*32 weights and 32 biases
output_layer_vals = {'weights':tf.Variable(tf.random_normal ([n_nodes_hl1+1,n_nodes_outl])) }

In [4]:
# user with 3706 ratings goes in
input_layer = tf.placeholder('float', [None, 3706])

# add a constant node to the first layer
# it needs to have the same shape as the input layer for me to be
# able to concatinate it later

input_layer_const = tf.fill( [tf.shape(input_layer)[0], 1] ,1.0  )
input_layer_concat =  tf.concat([input_layer, input_layer_const], 1)

# multiply output of input_layer wth a weight matrix 
layer_1 = tf.nn.sigmoid(tf.matmul(input_layer_concat, hidden_1_layer_vals['weights']))

# adding one bias node to the hidden layer
layer1_const = tf.fill( [tf.shape(layer_1)[0], 1] ,1.0  )
layer_concat =  tf.concat([layer_1, layer1_const], 1)

# multiply output of hidden with a weight matrix to get final output
output_layer = tf.matmul( layer_concat,output_layer_vals['weights'])

# output_true shall have the original shape for error calculations
output_true = tf.placeholder('float', [None, 3706])

# define our cost function
meansq =    tf.reduce_mean(tf.square(output_layer - output_true))
# define our optimizer

learn_rate = 0.1   # how fast the model should learn
optimizer = tf.train.AdagradOptimizer(learn_rate).minimize(meansq)

In [5]:
# initialising variables and starting the session
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# defining batch size, number of epochs and learning rate
batch_size = 100  # how many images to use together for training
hm_epochs =200    # how many times to go through the entire dataset
tot_users = X_train.shape[0] # total number of images

In [6]:
# running the model for a 200 epochs taking 100 users in batches
# total improvement is printed out after each epoch

for epoch in range(hm_epochs):
    epoch_loss = 0    # initializing error as 0
    
    for i in range(int(tot_users/batch_size)):
        epoch_x = X_train[ i*batch_size : (i+1)*batch_size ]
        _, c = sess.run([optimizer, meansq],\
               feed_dict={input_layer: epoch_x, \
               output_true: epoch_x})
        epoch_loss += c
        
    output_train = sess.run(output_layer, feed_dict={input_layer:X_train})
    output_test = sess.run(output_layer, feed_dict={input_layer:X_test})
        
    print('MSE train', MSE(output_train, X_train),'MSE test', MSE(output_test, X_test))      
    print('Epoch', epoch, '/', hm_epochs, 'loss:',epoch_loss)

MSE train 59.18676900600182 MSE test 58.46639464491769
Epoch 0 / 200 loss: 4044.6285552978516
MSE train 39.64310267008742 MSE test 38.72505068099756
Epoch 1 / 200 loss: 2331.7318801879883
MSE train 28.835596204435262 MSE test 28.040733640283474
Epoch 2 / 200 loss: 1632.8321571350098
MSE train 22.292538173099565 MSE test 21.667092933946513
Epoch 3 / 200 loss: 1225.2616271972656
MSE train 18.226457598807546 MSE test 17.751587227061087
Epoch 4 / 200 loss: 973.7096853256226
MSE train 15.527897946049057 MSE test 15.163832910418272
Epoch 5 / 200 loss: 813.8804092407227
MSE train 13.61424250537136 MSE test 13.377177554261115
Epoch 6 / 200 loss: 703.4661560058594
MSE train 12.205239096944625 MSE test 12.030698148975208
Epoch 7 / 200 loss: 624.1632089614868
MSE train 11.09787605330402 MSE test 10.9911187586128
Epoch 8 / 200 loss: 563.1522836685181
MSE train 10.204412749818175 MSE test 10.140432278631266
Epoch 9 / 200 loss: 514.9302706718445
MSE train 9.487894121686365 MSE test 9.475312792632826

MSE train 2.6102872435415327 MSE test 3.0439035953495086
Epoch 87 / 200 loss: 125.35692226886749
MSE train 2.5944286570259822 MSE test 3.028730080896134
Epoch 88 / 200 loss: 124.588019490242
MSE train 2.5788204065554967 MSE test 3.013790323473512
Epoch 89 / 200 loss: 123.82994854450226
MSE train 2.5633340891852585 MSE test 2.9989738808352606
Epoch 90 / 200 loss: 123.0861028432846
MSE train 2.548246149124488 MSE test 2.9846091273870052
Epoch 91 / 200 loss: 122.34780442714691
MSE train 2.533250287992639 MSE test 2.970548958228801
Epoch 92 / 200 loss: 121.6274664402008
MSE train 2.518249577085702 MSE test 2.9565524260822063
Epoch 93 / 200 loss: 120.91036474704742
MSE train 2.503597547730121 MSE test 2.9428865688562844
Epoch 94 / 200 loss: 120.19228446483612
MSE train 2.4894798684824275 MSE test 2.929570168238711
Epoch 95 / 200 loss: 119.49170923233032
MSE train 2.4761274688402968 MSE test 2.916762909297479
Epoch 96 / 200 loss: 118.82160651683807
MSE train 2.4627603635120425 MSE test 2.904

MSE train 1.8559137447089462 MSE test 2.354310437591753
Epoch 173 / 200 loss: 88.82133793830872
MSE train 1.85094830611323 MSE test 2.3502549804720827
Epoch 174 / 200 loss: 88.59145927429199
MSE train 1.8464640563946424 MSE test 2.346331620443487
Epoch 175 / 200 loss: 88.35449290275574
MSE train 1.8422137291298843 MSE test 2.3425498337938064
Epoch 176 / 200 loss: 88.1411645412445
MSE train 1.8380688441771826 MSE test 2.338803189231594
Epoch 177 / 200 loss: 87.93859851360321
MSE train 1.8339293736893925 MSE test 2.33505300078013
Epoch 178 / 200 loss: 87.74081194400787
MSE train 1.8297901519299022 MSE test 2.3312727333551844
Epoch 179 / 200 loss: 87.54355275630951
MSE train 1.8256279744575743 MSE test 2.327493264812756
Epoch 180 / 200 loss: 87.34578227996826
MSE train 1.8213031860222473 MSE test 2.323734351425725
Epoch 181 / 200 loss: 87.14727437496185
MSE train 1.8168282737141137 MSE test 2.3199806467042796
Epoch 182 / 200 loss: 86.94065701961517
MSE train 1.812406548651541 MSE test 2.3

In [7]:
# pick a user
sample_user = X_test.iloc[99,:]
#get the predicted ratings
sample_user_pred = sess.run(output_layer, feed_dict={input_layer:[sample_user]})

In [8]:
sample_user_pred

array([[ 5.461074  , -0.478554  ,  2.5884945 , ..., -0.4671824 ,
         1.9086571 , -0.29935086]], dtype=float32)